In [1]:
%pylab
import pickle
import os

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


QStandardPaths: wrong ownership on runtime directory /mnt/wslg/runtime-dir, 1000 instead of 0


# Figure 2

In [6]:
import pickle
f = open('DataFig2.pkl','rb')
Store = pickle.load(f)
f.close()

Ts0 = Store['Ts0']
Ts = Store['Ts']
Ratios0 = Store['Ratios0']
Ratios = Store['Ratios']
CEqs = Store['CEqs']
Alphas = Store['Alphas']
MTT = Store['MTT']
DTT = Store['DTT']
Iters = Store['Iters']
ItersCL = Store['ItersCL']
AllCostsCL = Store['AllCostsCL']
gm = Store['gm']
qs = Store['qs']



In [5]:
import seaborn as sns
import itertools as it
import matplotlib.colors as mcolors
from scipy.signal import savgol_filter
from scipy.stats.mstats import gmean


jmp = 3
jmpx = 1

widths = [8., 8., 0.5]
#widths = [8., 8.]
heights = [1, 1]

f = figure(figsize=(12,10))
spec = f.add_gridspec(ncols=3, nrows=2, width_ratios=widths)

a1 = f.add_subplot(spec[0,0])
t0 = 15
t1 = len(Ts0)

tids = array([24,27,30,33,36,39,42,45])

cm1 = mcolors.LinearSegmentedColormap.from_list('RedToBlue',['b','r'])
cm1r = mcolors.LinearSegmentedColormap.from_list('RedToBlueR',['r','b'])
norm1 = mcolors.Normalize(vmin=tids[0], vmax=tids[-1])
cmap = mpl.cm.viridis_r
cmap = cm1
smap = mpl.cm.ScalarMappable(norm=norm1, cmap=cmap)

#for tid in range(t0,t1,jmp):
for tid in tids:
    Ts0[tid][0] = 1.e-20
    a1.plot(Ts0[tid][0::jmpx]/Ratios0[tid], CEqs[tid][0::jmpx]/CEqs[tid][0], color=smap.to_rgba(tid), lw=2, label=r'$'+str(around(log10(Ratios0[tid]),0))+'$')
    
    ef = 0.5
    els = -int(len(Ts[tid])*ef)
    pp = polyfit(Ts[tid][els:], log(CEqs[tid][els:]), 1)


a1.plot(arange(ItersCL), AllCostsCL/AllCostsCL[0], lw=2, ls=':', color='w')

a1.set_xlim(8.e-1,2.e4)
a1.set_ylim(5.e-6,1.e1)
a1.set_xscale('log')
a1.set_yscale('log')
a1.set_xlabel('Training steps', size=20)
a1.set_ylabel('Equilibrium MSE', size=20)
a1.xaxis.set_tick_params(labelsize=16)
a1.xaxis.set_ticks([1.e1,1.e3])
a1.xaxis.set_ticklabels([r'$10$',r'$10^{3}$'])
a1.yaxis.set_tick_params(labelsize=16)
a1.yaxis.set_ticks([1.e-5,1.e-4,1.e-3,1.e-2,1.e-1,1.e0,1.e1])
a1.yaxis.set_ticklabels([r'$10^{-5}$',r'$10^{-4}$',r'$10^{-3}$',r'$10^{-2}$',r'$10^{-1}$',r'$1$',r'$10$'])

a1.text(-0.17, 1.1, '(a)', transform=a1.transAxes, fontsize=26, va='top', ha='right')


a2 = f.add_subplot(spec[0,1])
for tid in tids:
    a2.plot(Ts0[tid][::jmpx],CEqs[tid][::jmpx]/CEqs[tid][0], color=smap.to_rgba(tid), lw=2)

a2.set_xlim(1.e-1,3.e5)
a2.set_ylim(5.e-6,1.e1)
a2.set_xscale('log')
a2.set_yscale('log')
a2.set_xlabel('Real time', size=20)
a2.xaxis.set_tick_params(labelsize=16)
a2.xaxis.set_ticks([1.e-1,1.e1,1.e3,1.e5])
a2.xaxis.set_ticklabels([r'$10^{-1}$',r'$10$',r'$10^{3}$',r'$10^{5}$'])
a2.yaxis.set_ticklabels([])

a2.text(-0.05, 1.1, '(b)', transform=a2.transAxes, fontsize=26, va='top', ha='right')

acb = f.add_subplot(spec[0,2])
norm2 = mcolors.LogNorm(vmin=1/Ratios0[tids[-1]], vmax=1/Ratios0[tids[0]])
cb1 = mpl.colorbar.ColorbarBase(acb, cmap=cm1r,
                                norm=norm2,
                                orientation='vertical')
cb1.set_label(r'$\mathcal{R}$', fontsize=18)
cb1.ax.tick_params(labelsize=16)
cb1.ax.yaxis.set_ticks([1.e-3,1.e-2,1.e-1,1,10,100,1000,10000])
cb1.ax.yaxis.set_ticklabels([r'$10^{-3}$',r'$10^{-2}$',r'$10^{-1}$',r'$1$', r'$10$',r'$10^{2}$',r'$10^{3}$',r'$10^{4}$'])


a3 = f.add_subplot(spec[1,0])

cm1 = mcolors.LinearSegmentedColormap.from_list('RedToBlue',['b','r'])
cm1r = mcolors.LinearSegmentedColormap.from_list('RedToBlueR',['r','b'])
norm1 = mcolors.Normalize(vmin=log10(Ratios[-1]), vmax=log10(Ratios[0]))
cmap = cm1
smap = mpl.cm.ScalarMappable(norm=norm1, cmap=cmap)

Aid = 0
Tid = Iters[Alphas[Aid]]


a,b,c = a3.errorbar(1./Ratios, gm, yerr=[gm-qs[0], qs[1]-gm], lw=3, label=r'$'+str(Alphas[Aid])+'$', color='k', fmt='o', ms=0)
colors = smap.to_rgba(log10(Ratios))
c[0].set_color(colors)
a3.scatter(1./Ratios, gm, color=colors, s=30)


a3.set_xscale('log')
a3.set_yscale('log')
a3.set_xlabel(r'$\mathcal{R}$', size=20)
a3.set_ylabel(r'$\mathrm{Trained\ equilibrium\ MSE}$', size=20)
#a3.set_ylabel(r'$\mathrm{Trained\ error}$', size=20)
a3.xaxis.set_tick_params(labelsize=16)
a3.xaxis.set_ticks([1.e-3,1.,1.e3,1.e6,1.e9,1.e12])
a3.xaxis.set_ticklabels([r'$10^{-3}$',r'$1$',r'$10^{3}$',r'$10^{6}$',r'$10^{9}$',r'$10^{12}$'])
a3.yaxis.set_tick_params(labelsize=16)

a3.text(-0.17, 1.1, '(c)', transform=a3.transAxes, fontsize=26, va='top', ha='right')


a4 = f.add_subplot(spec[1,1])

cm1 = mcolors.LinearSegmentedColormap.from_list('RedToBlue',['b','r'])
cm1r = mcolors.LinearSegmentedColormap.from_list('RedToBlueR',['r','b'])
norm1 = mcolors.Normalize(vmin=log10(Ratios[-1]), vmax=log10(Ratios[0]))
cmap = cm1
smap = mpl.cm.ScalarMappable(norm=norm1, cmap=cmap)

TrainTimes = array([Ts[i][-1] for i in range(len(Ts))])
a,b,c = a4.errorbar(1./Ratios, MTT, DTT, lw=3, label=r'$'+str(Alphas[Aid])+'$', color='k', fmt='o', ms=0)
colors = smap.to_rgba(log10(Ratios))
c[0].set_color(colors)
a4.scatter(1./Ratios, MTT, color=colors, marker='o', s=30)

i0 = 0; i1 = 6
a4.plot(1./Ratios[i0:i1], 50*Ratios[i0:i1]**1., 'k--')
i0 = 12; i1 = 34
a4.plot(1./Ratios[i0:i1], 60*Ratios[i0:i1]**0.5, 'k--')
a4.set_xscale('log')
a4.xaxis.set_ticks([1.e-3,1.,1.e3,1.e6,1.e9,1.e12])
a4.xaxis.set_ticklabels([r'$10^{-3}$',r'$1$',r'$10^{3}$',r'$10^{6}$',r'$10^{9}$',r'$10^{12}$'])
a4.set_yscale('log')
#a4.yaxis.set_ticks([1.e-3,1.,1.e3,1.e6,1.e9,1.e12])
#a4.yaxis.set_ticklabels([r'$10^{-3}$',r'$1$',r'$10^{3}$',r'$10^{6}$',r'$10^{9}$',r'$10^{12}$'])

a4.set_xlabel(r'$\mathcal{R}$', size=20)
a4.set_ylabel(r'$\mathrm{Training\ time}$', size=20)
a4.xaxis.set_tick_params(labelsize=16)
a4.yaxis.set_tick_params(labelsize=16)
a4.yaxis.set_ticks([1.e-3,1.e-1,1.e1,1.e3,1.e5])
a4.yaxis.set_ticklabels([r'$10^{-3}$',r'$10^{-1}$',r'$10$',r'$10^{3}$',r'$10^{5}$'])

a4.text(8.e-4,6.e1,r'$\mathcal{R}^{-1}$', size=16, color='k')
a4.text(5.e3,5.e-3,r'$\mathcal{R}^{-1/2}$', size=16, color='k')


a4.text(-0.05, 1.1, '(d)', transform=a4.transAxes, fontsize=26, va='top', ha='right')


f.subplots_adjust(left=0.12, bottom=0.1,top=0.96, wspace=0.48, hspace=0.3)


# Figure 3

In [2]:
import pickle
f = open('DataFig3.pkl','rb')
Store = pickle.load(f)
f.close()


Ratios = Store['Ratios']
Ratios1 = Store['Ratios1']
NY = Store['NY']
NDY = Store['NDY']
NC = Store['NC']
MES = Store['MES']
DES = Store['DES']

In [3]:
import seaborn as sns
import itertools as it
import matplotlib.colors as mcolors
from scipy.signal import savgol_filter
from scipy.stats.mstats import gmean


jmp = 3
jmpx = 1

widths = [8., 8.]
heights = [1, 1]

f = figure(figsize=(10,6))
spec = f.add_gridspec(ncols=2, nrows=1, width_ratios=widths)

a1 = f.add_subplot(spec[0,0])


cm1 = mcolors.LinearSegmentedColormap.from_list('RedToBlue',['b','r'])
cm1r = mcolors.LinearSegmentedColormap.from_list('RedToBlueR',['r','b'])
norm1 = mcolors.Normalize(vmin=log10(Ratios[-1]), vmax=log10(Ratios[0]))
cmap = cm1
smap = mpl.cm.ScalarMappable(norm=norm1, cmap=cmap)


a,b,c = a1.errorbar(1./Ratios, 1./NY, NDY/sqrt(NC-1)/NY**2, lw=3, fmt='o', ms=0)
colors = smap.to_rgba(log10(Ratios))
c[0].set_color(colors)
a1.scatter(1./Ratios, 1./NY, color=colors, s=30)
a1.plot(1./Ratios[8:-2], 1./(5*Ratios[8:-2]**(0.5)),'k--')
a1.text(1.e2,5.e1,r'$\mathcal{R}^{1/2}$', size=16, color='k')


a1.set_xlim(1.e-1,2.e12)
a1.set_ylim(5.e-2,1.e5)
a1.set_xscale('log')
a1.set_yscale('log')
a1.set_xlabel(r'$\mathcal{R}$', size=20)
a1.set_ylabel(r'$\mathrm{Oscillation\ frequency}$', size=20)
a1.xaxis.set_tick_params(labelsize=16)
a1.yaxis.set_tick_params(labelsize=16)
a1.yaxis.set_ticks([1.e-1,1,10,1.e2,1.e3,1.e4,1.e5])
a1.yaxis.set_ticklabels([r'$10^{-1}$',r'$1$', r'$10$',r'$10^{2}$',r'$10^{3}$',r'$10^{4}$',r'$10^{5}$'])
#a1.legend(loc=0, fontsize=16, title=r'$\log_{10}(\alpha/\gamma)$', title_fontsize=16)

a1.text(-0.13, 1.1, '(a)', transform=a1.transAxes, fontsize=26, va='top', ha='right')


a2 = f.add_subplot(spec[0,1])


cm1 = mcolors.LinearSegmentedColormap.from_list('RedToBlue',['b','r'])
cm1r = mcolors.LinearSegmentedColormap.from_list('RedToBlueR',['r','b'])
norm1 = mcolors.Normalize(vmin=log10(Ratios1[0]), vmax=log10(Ratios1[-1]))
cmap = cm1r
smap = mpl.cm.ScalarMappable(norm=norm1, cmap=cmap)

a,b,c = a2.errorbar(Ratios1[::2], 1./MES.T[0][::2], DES.T[0][::2]/MES.T[0][::2]**2, lw=2, color='k', label=r'$\lambda_1^{-1}$', fmt='o', ms=0)
colors = smap.to_rgba(log10(Ratios1[::2]))
c[0].set_color(colors)
a2.scatter(Ratios1[::2], 1./MES.T[0][::2], color=colors, s=30, marker='o')

a,b,c = a2.errorbar(Ratios1[::2], 1./MES.T[1][::2], DES.T[1][::2]/MES.T[1][::2]**2, lw=2, color='k', label=r'$\lambda_2^{-1}$', fmt='o', ms=0)
colors = smap.to_rgba(log10(Ratios1[::2]))
c[0].set_color(colors)
a2.scatter(Ratios1[::2], 1./MES.T[1][::2], color=colors, s=30, marker='s')

a,b,c = a2.errorbar(Ratios1[::2], 1./MES.T[2][::2], DES.T[2][::2]/MES.T[2][::2]**2, lw=2, color='k', label=r'$\lambda_3^{-1}$', fmt='o', ms=0)
colors = smap.to_rgba(log10(Ratios1[::2]))
c[0].set_color(colors)
a2.scatter(Ratios1[::2], 1./MES.T[2][::2], color=colors, s=30, marker='^')

a2.plot(Ratios1[32:], Ratios1[32:]/10. ,'k--')
a2.text(1.e2,1.e2,r'$\mathcal{R}^1$', size=16, color='k')



a2.set_xscale('log')
a2.set_yscale('log')
a2.set_xlabel(r'$\mathcal{R}$', size=20)
a2.set_ylabel(r'$\mathrm{Flatness}$', size=20)
a2.xaxis.set_tick_params(labelsize=16)
a2.xaxis.set_ticks([0.1,10,1.e3])
a2.xaxis.set_ticklabels([r'$10^{-1}$', r'$10$',r'$10^{3}$'])
a2.yaxis.set_tick_params(labelsize=16)
a2.yaxis.set_ticks([1,10,1.e2,1.e3])
a2.yaxis.set_ticklabels([r'$1$', r'$10$',r'$10^{2}$',r'$10^{3}$'])

from matplotlib.patches import Patch
from matplotlib.lines import Line2D

legend_elements = [Line2D([0], [0], marker='o', color='w', label=r'$\lambda_1^{-1}$', markerfacecolor='b', markersize=15),
                   Line2D([0], [0], marker='s', color='w', label=r'$\lambda_2^{-1}$', markerfacecolor='b', markersize=15),
                   Line2D([0], [0], marker='^', color='w', label=r'$\lambda_3^{-1}$', markerfacecolor='b', markersize=15)]
a2.legend(handles=legend_elements, loc=0, fontsize=18, fancybox=True)


a2.text(-0.0, 1.1, '(b)', transform=a2.transAxes, fontsize=26, va='top', ha='right')



subplots_adjust(left=0.10, bottom=0.12)
f.tight_layout()

/tmp/ipykernel_478/2507102056.py:27: RuntimeWarning: invalid value encountered in sqrt
  a,b,c = a1.errorbar(1./Ratios, 1./NY, NDY/sqrt(NC-1)/NY**2, lw=3, fmt='o', ms=0)
/tmp/ipykernel_478/2507102056.py:27: RuntimeWarning: invalid value encountered in true_divide
  a,b,c = a1.errorbar(1./Ratios, 1./NY, NDY/sqrt(NC-1)/NY**2, lw=3, fmt='o', ms=0)
